In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [2]:
import tensorflow as tf
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.2.0-rc4-8-g2b96f3662b 2.2.0


In [3]:
# import tensorflow v2 - disable eager execution to build explicit graph for deployment
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(1)
tf.debugging.set_log_device_placement(True)
tf.compat.v1.disable_eager_execution()

# import other libraries
import pandas as pd
import numpy as np

from combined_estimator import CombinedEstimator
from model_fns import * # all model_fns are using python closures

In [4]:
LEARNING_RATE=0.001
BATCH_SIZE = 256
TRAIN_MAX_STEPS=None
STEPS = None
NUM_EPOCHS = 1 #TODO: Change this
LEAKY_RELU_ALPHA = 0.1

In [5]:
TRAIN_FILE = 'sample_train.csv'#O2D_Jul10_train_data.csv'
EVAL_FILE = 'sample_test.csv'#O2D_Jul10_test_data.csv'
MODEL_DIR="o2d"
SAVE_CHECKPOINT_SECS = 900 # once in 15 minutes
SAVE_SUMMARY_STEPS = 4000 # on 4000 epochs
KEEP_CHECKPOINT_MAX = 5 # keep small number to not explode disk space

In [6]:
features_fm_regressor = ['banner_factor','weekday','hour','fm_avg_Rz_1H','fm_avg_Rid_1Day','fm_avg_Rid_hour_30D','rest_zone_rain_mode__2H','fm_avg_Rz_dow_hour_7D','fm_avg_Rz_3H','fm_avg_Rz_dow_hour_30D','fm_avg_Rz_3D','fm_avg_Rz_1D','fm_avg_Rh_1D','fm_avg_Rh_hour_3D','fm_avg_Rh_dow_hour_7D']
features_lm_regressor  = ['rant_cst_distance','p2d_avg_Rz_3D','p2d_avg_Rz_1H','p2d_avg_Cz_3D','p2d_avg_Rid_20D','p2d_avg_Rid_1H','p2d_avg_Cz_1H','p2d_avg_Rid_3D','r2d_avg_Cg_7D','p2d_avg_Ch_20D','p2d_sd_Ch_20D','rest_zone_rain_mode__2H','cust_zone_rain_mode__2H','embedding_dayofweek_LM_0','embedding_dayofweek_LM_1','embedding_dayofweek_LM_2','embedding_hour_LM_0','embedding_hour_LM_1','embedding_hour_LM_2','embedding_hour_LM_3','embedding_hour_LM_4','embedding_city_LM_0','embedding_city_LM_1','embedding_city_LM_2','embedding_city_LM_3','embedding_city_LM_4','embedding_Rz_LM_0','embedding_Rz_LM_1','embedding_Rz_LM_2','embedding_Rz_LM_3','embedding_Rz_LM_4','embedding_Rz_LM_5','embedding_Rz_LM_6','embedding_Rz_LM_7','embedding_Rz_LM_8','embedding_Rz_LM_9','embedding_Rz_LM_10','embedding_Rz_LM_11','embedding_Rz_LM_12','embedding_Rz_LM_13','embedding_Rz_LM_14','embedding_Rz_LM_15','embedding_Rz_LM_16','embedding_Rz_LM_17','embedding_Rz_LM_18','embedding_Rz_LM_19','embedding_Rz_LM_20','embedding_Rz_LM_21','embedding_Rz_LM_22','embedding_Rz_LM_23','embedding_Rz_LM_24']
features_o2a_regressor = ['banner_factor','weekday','hour','o2as_avg_res_1h','o2a_zoneid_last15mins_averages','o2a_zoneid_last5mins_averages','o2as_avg_Rid_hour_dow_30d','o2as_avg_Rz_hour_dow_30d','o2as_avg_Rz_hour_dow_9d','o2as_avg_Rz_hour_dow_lag','rest_zone_rain_mode__2H','o2as_avg_Rid_hour_3D','o2as_avg_Rz_dow_30D','o2as_avg_Rz_7D','o2as_avg_Rh_dow_7D','o2as_avg_Rh_7D','o2as_avg_Rid_dow_hour_7D']
features_o2d_regressor = ['bill_amount','banner_factor','hour','rant_cst_distance','o2as_avg_res_1h','o2a_zoneid_last15mins_averages','o2a_zoneid_last5mins_averages','o2as_avg_Rid_hour_dow_30d','o2as_avg_Rz_hour_dow_30d','o2as_avg_Rz_hour_dow_9d','o2as_avg_Rz_hour_dow_lag','o2as_avg_Rid_hour_3D','o2as_avg_Rz_dow_30D','o2as_avg_Rz_7D','o2as_avg_Rh_dow_7D','o2as_avg_Rh_7D','o2as_avg_Rid_dow_hour_7D','rest_zone_rain_mode__2H','cust_zone_rain_mode__2H','fm_avg_Rz_1H','fm_avg_Rid_1Day','fm_avg_Rz_dow_hour_7D','fm_avg_Rz_3H','fm_avg_Rz_dow_hour_30D','fm_avg_Rz_3D','fm_avg_Rz_1D','fm_avg_Rh_1D','fm_avg_Rh_hour_3D','fm_avg_Rh_dow_hour_7D','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','order_item_count','order_total_quantity','rant_active_orders','rant_banner_factor','items_past','o2p_value_res_lag1','item_o2p_lag','item_o2p_avg','item_slot_o2p_avg','item_slot_o2p_avg_1D','o2p_avg_1H','avg_item','high_item','rare_item','lag_delta_I','lag_delta_II','item_lag_delta_I','rest_placed_order_count__1h','rant_delta','o2mf_value_res_lag1','o2mf_avg_1H','p2d_avg_Rz_3D','p2d_avg_Rz_1H','p2d_avg_Cz_3D','p2d_avg_Rid_20D','p2d_avg_Rid_1H','p2d_avg_Cz_1H','p2d_avg_Rid_3D','r2d_avg_Cg_7D','p2d_avg_Ch_20D','p2d_sd_Ch_20D','embedding_hour_LM_0','embedding_hour_LM_1','embedding_hour_LM_2','embedding_hour_LM_3','embedding_hour_LM_4','embedding_city_LM_0','embedding_city_LM_1','embedding_city_LM_2','embedding_city_LM_3','embedding_city_LM_4','embedding_Rz_LM_0','embedding_Rz_LM_1','embedding_Rz_LM_2','embedding_Rz_LM_3','embedding_Rz_LM_4','embedding_Rz_LM_5','embedding_Rz_LM_6','embedding_Rz_LM_7','embedding_Rz_LM_8','embedding_Rz_LM_9','embedding_Rz_LM_10','embedding_Rz_LM_11','embedding_Rz_LM_12','embedding_Rz_LM_13','embedding_Rz_LM_14','embedding_Rz_LM_15','embedding_Rz_LM_16','embedding_Rz_LM_17','embedding_Rz_LM_18','embedding_Rz_LM_19','embedding_Rz_LM_20','embedding_Rz_LM_21','embedding_Rz_LM_22','embedding_Rz_LM_23','embedding_Rz_LM_24','embedding_hour_O2P_0','embedding_hour_O2P_1','embedding_hour_O2P_2','embedding_hour_O2P_3','embedding_hour_O2P_4','embedding_week_O2P_0','embedding_week_O2P_1','embedding_week_O2P_2','embedding_rant_O2P_0','embedding_rant_O2P_1','embedding_rant_O2P_2','embedding_rant_O2P_3','embedding_rant_O2P_4','embedding_rant_O2P_5','embedding_rant_O2P_6','embedding_rant_O2P_7','embedding_rant_O2P_8','embedding_rant_O2P_9','embedding_city_O2P_0','embedding_city_O2P_1','embedding_city_O2P_2','embedding_city_O2P_3','embedding_city_O2P_4']
features_o2p_regressor = ['bill_amount','banner_factor','hour','weekday','order_item_count','order_total_quantity','rant_active_orders','rant_banner_factor','items_past','o2p_value_res_lag1','item_o2p_lag','item_o2p_avg','item_slot_o2p_avg','item_slot_o2p_avg_1D','o2p_avg_1H','avg_item','high_item','rare_item','lag_delta_I','lag_delta_II','item_lag_delta_I','rest_placed_order_count__1h','rant_delta','o2mf_value_res_lag1','o2mf_avg_1H','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','rest_zone_rain_mode__2H','embedding_hour_O2P_0','embedding_hour_O2P_1','embedding_hour_O2P_2','embedding_hour_O2P_3','embedding_hour_O2P_4','embedding_week_O2P_0','embedding_week_O2P_1','embedding_week_O2P_2','embedding_rant_O2P_0','embedding_rant_O2P_1','embedding_rant_O2P_2','embedding_rant_O2P_3','embedding_rant_O2P_4','embedding_rant_O2P_5','embedding_rant_O2P_6','embedding_rant_O2P_7','embedding_rant_O2P_8','embedding_rant_O2P_9','embedding_city_O2P_0','embedding_city_O2P_1','embedding_city_O2P_2','embedding_city_O2P_3','embedding_city_O2P_4']
features_wt_regressor = ['banner_factor','weekday','hour','ar2p_avg_res_hour_20d','ar2p_avg_res_20d','ar2p_avg_res_1h','rest_zone_rain_mode__2H']

output_features = ['O2D','O2D_beef','Ar2P','O2As','FM','O2P','P2D']

input_features = list(set(features_fm_regressor + features_lm_regressor + features_o2a_regressor + features_o2d_regressor + features_o2p_regressor + features_wt_regressor))

In [7]:
def train_input_fn():
    
    #select both input and output features during training
    dataset = tf.data.experimental.make_csv_dataset(
      file_pattern=TRAIN_FILE,
      batch_size=BATCH_SIZE,
      num_epochs=NUM_EPOCHS,
      ignore_errors=True, 
      select_columns=input_features + output_features, 
      column_defaults=['float64' for _ in range(len(input_features + output_features))],
      shuffle=False)
    
    return dataset

In [8]:
train_input_fn()

<PrefetchDataset shapes: OrderedDict([(order_item_count, (None,)), (rant_delta, (None,)), (embedding_rant_O2P_2, (None,)), (o2a_zoneid_last15mins_averages, (None,)), (ar2p_avg_res_20d, (None,)), (rest_placed_order_count__1h, (None,)), (embedding_hour_LM_0, (None,)), (embedding_Rz_LM_13, (None,)), (avg_item, (None,)), (embedding_week_O2P_0, (None,)), (embedding_city_LM_2, (None,)), (embedding_hour_LM_3, (None,)), (items_past, (None,)), (high_item, (None,)), (o2mf_value_res_lag1, (None,)), (embedding_hour_O2P_3, (None,)), (fm_avg_Rh_dow_hour_7D, (None,)), (embedding_Rz_LM_0, (None,)), (embedding_Rz_LM_21, (None,)), (fm_avg_Rz_3H, (None,)), (o2p_value_res_lag1, (None,)), (p2d_avg_Cz_1H, (None,)), (embedding_city_LM_3, (None,)), (embedding_rant_O2P_0, (None,)), (o2as_avg_Rz_7D, (None,)), (rant_active_orders, (None,)), (embedding_rant_O2P_4, (None,)), (cust_zone_rain_mode__2H, (None,)), (embedding_hour_LM_2, (None,)), (embedding_Rz_LM_23, (None,)), (p2d_sd_Ch_20D, (None,)), (embedding_city_

In [9]:
def eval_input_fn():
    
    #select both input and output features during evaluation
    dataset = tf.data.experimental.make_csv_dataset(
      file_pattern=EVAL_FILE,
      batch_size=BATCH_SIZE,
      num_epochs=1, #only once should evaluate
      ignore_errors=True, 
      select_columns=input_features + output_features, 
      column_defaults=['float64' for _ in range(len(input_features + output_features))],
      shuffle=False)
    
    return dataset

In [10]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'), communication = CollectiveCommunication.AUTO


In [11]:
# this defines at what configuration does training run
rc = tf.estimator.RunConfig(train_distribute=strategy,eval_distribute=strategy,
                            save_checkpoints_secs = SAVE_CHECKPOINT_SECS,
                           model_dir = MODEL_DIR,
                           save_summary_steps = SAVE_SUMMARY_STEPS,
                           keep_checkpoint_max = KEEP_CHECKPOINT_MAX)

INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


In [ ]:
o2a_estimator = tf.estimator.Estimator(
    model_fn=o2a_model_fn(features_o2d_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                         LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

fm_estimator = tf.estimator.Estimator(
    model_fn=fm_model_fn(features_fm_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

wt_estimator = tf.estimator.Estimator(
    model_fn=wt_model_fn(features_wt_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

o2p_estimator = tf.estimator.Estimator(
    model_fn=o2p_model_fn(features_o2p_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

lm_estimator = tf.estimator.Estimator(
    model_fn=lm_model_fn(features_lm_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

o2d_estimator = tf.estimator.Estimator(
    model_fn=o2d_model_fn(features_o2d_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

o2d_beef_estimator = tf.estimator.Estimator(
    model_fn=o2d_beef_model_fn(features_o2d_regressor, LEAKY_RELU_ALPHA, LEARNING_RATE,
                        LOSS='custom'),
    model_dir=MODEL_DIR,
    config=rc)

In [13]:
estimator = CombinedEstimator([o2a_estimator, fm_estimator, wt_estimator, o2p_estimator, 
                               lm_estimator, o2d_estimator, o2d_beef_estimator],
                              ['O2As','FM','Ar2P','O2P','P2D','O2D','O2D_beef'], 
                              MODEL_DIR)

In [19]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=TRAIN_MAX_STEPS)
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn)

tf.estimator.train_and_evaluate(estimator, train_spec=train_spec, eval_spec=eval_spec)

TypeError: in user code:

    /usr/lib/environs/e-a-2019.03-py-3.7.3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/estimator.py:1170 _call_model_fn  *
        model_fn_results = self._model_fn(features=features, **kwargs)
    /media/ephemeral0/combined_estimator.py:39 model_fn  *
        if spec.train_op:
    /usr/lib/environs/e-a-2019.03-py-3.7.3/lib/python3.7/site-packages/tensorflow/python/autograph/operators/control_flow.py:924 if_stmt
        basic_symbol_names, composite_symbol_names)
    /usr/lib/environs/e-a-2019.03-py-3.7.3/lib/python3.7/site-packages/tensorflow/python/autograph/operators/control_flow.py:962 tf_if_stmt
        error_checking_orelse)
    /usr/lib/environs/e-a-2019.03-py-3.7.3/lib/python3.7/site-packages/tensorflow/python/util/deprecation.py:507 new_func
        return func(*args, **kwargs)
    /usr/lib/environs/e-a-2019.03-py-3.7.3/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:1177 cond
        return cond_v2.cond_v2(